In [1]:
import numpy as np
import tensorflow as tf

In [2]:
def khatrirao_product(tensor1, tensor2):
    i0 = tf.constant(1)
    prod0 = tf.multiply(tensor1[0, :], tensor2)
    _, khprod = tf.while_loop(lambda i, m: tf.less(i, tf.shape(tensor1)[0]),
                              lambda i, m: [i+1, tf.concat([m, tf.multiply(tensor1[i, :], tensor2)], 0)],
                              loop_vars=[i0, prod0],
                              shape_invariants=[i0.get_shape(), tf.TensorShape([None, None])])
    return khprod

In [3]:
def rank3tensor_decomposition_ALS(matrix, k, alpha=0.01, nbiter=1000):
    dim0, dim1, dim2 = matrix.shape

    #X = tf.placeholder(tf.float32, shape=(None, None, None), name='X')
    #X_shape = tf.shape(X)
    #Xdim0, Xdim1, Xdim2 = X_shape[0], X_shape[1], X_shape[2]
    X = tf.placeholder(tf.float32, shape=(dim0, dim1, dim2), name='X')
    
    X0 = tf.reshape(tf.transpose(X, perm=(0, 2, 1)), shape=(dim0, dim1*dim2))
    X1 = tf.reshape(tf.transpose(X, perm=(1, 2, 0)), shape=(dim1, dim0*dim2))
    X2 = tf.reshape(tf.transpose(X, perm=(2, 0, 1)), shape=(dim2, dim0*dim1))
    
    A = tf.Variable(initial_value=tf.random_normal([dim0, k]), name='A')
    B = tf.Variable(initial_value=tf.random_normal([dim1, k]), name='B')
    C = tf.Variable(initial_value=tf.random_normal([dim2, k]), name='C')

    costA = tf.reduce_sum(tf.abs(X0 - tf.matmul(A, tf.transpose(khatrirao_product(C, B)))))
    costB = tf.reduce_sum(tf.abs(X1 - tf.matmul(B, tf.transpose(khatrirao_product(C, A)))))
    costC = tf.reduce_sum(tf.abs(X2 - tf.matmul(C, tf.transpose(khatrirao_product(B, A)))))
    
    trainA = tf.train.GradientDescentOptimizer(alpha).minimize(costA)
    trainB = tf.train.GradientDescentOptimizer(alpha).minimize(costB)
    trainC = tf.train.GradientDescentOptimizer(alpha).minimize(costC)
    
    init = tf.global_variables_initializer()
    
    sess = tf.Session()
    sess.run(init)

    for _ in range(nbiter):
        sess.run(trainA, feed_dict={X: x})
        sess.run(trainB, feed_dict={X: x})
        sess.run(trainC, feed_dict={X: x})

    return sess.run((A, B, C), feed_dict={X: x})

In [4]:
def rank3tensor_decomposition_ALS_closed(matrix, k, nbiter=1000):
    dim0, dim1, dim2 = matrix.shape

    #X = tf.placeholder(tf.float32, shape=(None, None, None), name='X')
    #X_shape = tf.shape(X)
    #Xdim0, Xdim1, Xdim2 = X_shape[0], X_shape[1], X_shape[2]
    X = tf.placeholder(tf.float32, shape=(dim0, dim1, dim2), name='X')
    
    X0 = tf.reshape(tf.transpose(X, perm=(0, 2, 1)), shape=(dim0, dim1*dim2))
    X1 = tf.reshape(tf.transpose(X, perm=(1, 2, 0)), shape=(dim1, dim0*dim2))
    X2 = tf.reshape(tf.transpose(X, perm=(2, 0, 1)), shape=(dim2, dim0*dim1))
    
    A = tf.Variable(initial_value=tf.random_normal([dim0, k]), name='A')
    B = tf.Variable(initial_value=tf.random_normal([dim1, k]), name='B')
    C = tf.Variable(initial_value=tf.random_normal([dim2, k]), name='C')

    for _ in range(nbiter):
        A = tf.matmul(tf.matmul(X0, khatrirao_product(C, B)), 
                      tf.multiply(tf.matmul(tf.transpose(C), C), tf.matmul(tf.transpose(B), B)))
        B = tf.matmul(tf.matmul(X1, khatrirao_product(C, A)), 
                      tf.multiply(tf.matmul(tf.transpose(C), C), tf.matmul(tf.transpose(A), A)))
        C = tf.matmul(tf.matmul(X2, khatrirao_product(B, A)), 
                      tf.multiply(tf.matmul(tf.transpose(B), B), tf.matmul(tf.transpose(A), A)))
        
    init = tf.global_variables_initializer()
    
    sess = tf.Session()
    sess.run(init)
    
    return sess.run((A, B, C), feed_dict={X: x})

# Test: Method 1

In [18]:
# x1 = np.array([[1, 4, 7, 10], [2, 5, 8, 11], [3, 6, 9, 12]], dtype=np.float32)
# x2 = np.array([[13, 16, 19, 22], [14, 17, 20, 23], [15, 18, 21, 24]], dtype=np.float32)
# x = np.zeros((3, 4, 2))
# x[:, :, 0] = x1
# x[:, :, 1] = x2
x1 = np.array([[1, 0], [0, 1]], dtype=np.float32)
x2 = np.array([[0, -1], [1, 0]], dtype=np.float32)
x = np.zeros((2, 2, 2))
x[:, :, 0] = x1
x[:, :, 1] = x2

In [20]:
A, B, C = rank3tensor_decomposition_ALS(x, k=2, nbiter=10000)

In [26]:
B

array([[-0.3706024,  0.364294 ],
       [ 1.1831077,  1.1708362]], dtype=float32)

In [21]:
mat = np.zeros((A.shape[0], B.shape[0], C.shape[0]))

In [28]:
from itertools import product

for i, j, k in product(range(A.shape[0]), range(B.shape[0]), range(C.shape[0])):
    print('{}, {}, {}'.format(i, j, k))
    mat[i, j, k] = sum(A[i, alpha]*B[j, alpha]*C[k, alpha] for alpha in range(2))

0, 0, 0
0, 0, 1
0, 1, 0
0, 1, 1
1, 0, 0
1, 0, 1
1, 1, 0
1, 1, 1


In [29]:
mat

array([[[ 1.02457631,  0.00868315],
        [-0.05918944, -0.97800356]],

       [[-0.00624643, -0.09471185],
        [ 1.01790738,  0.00707337]]])

In [10]:
x

array([[[ 1., 13.],
        [ 4., 16.],
        [ 7., 19.],
        [10., 22.]],

       [[ 2., 14.],
        [ 5., 17.],
        [ 8., 20.],
        [11., 23.]],

       [[ 3., 15.],
        [ 6., 18.],
        [ 9., 21.],
        [12., 24.]]])

# Method 2

In [11]:
A, B, C = rank3tensor_decomposition_ALS_closed(x, k=5, nbiter=1)

In [12]:
mat = np.zeros((A.shape[0], B.shape[0], C.shape[0]))

In [13]:
from itertools import product

for i, j, k in product(range(A.shape[0]), range(B.shape[0]), range(C.shape[0])):
    mat[i, j, k] = sum(A[i, alpha]*B[j, alpha]*C[k, alpha] for alpha in range(5))

In [14]:
mat

array([[[inf, inf],
        [inf, inf],
        [inf, inf],
        [inf, inf]],

       [[inf, inf],
        [inf, inf],
        [inf, inf],
        [inf, inf]],

       [[inf, inf],
        [inf, inf],
        [inf, inf],
        [inf, inf]]])

In [15]:
x

array([[[ 1., 13.],
        [ 4., 16.],
        [ 7., 19.],
        [10., 22.]],

       [[ 2., 14.],
        [ 5., 17.],
        [ 8., 20.],
        [11., 23.]],

       [[ 3., 15.],
        [ 6., 18.],
        [ 9., 21.],
        [12., 24.]]])

In [16]:
A, B, C

(array([[  42.086662,  250.71745 ,  -61.108463,  -45.05388 , -435.62628 ],
        [  44.6645  ,  264.2911  ,  -64.514694,  -37.411827, -475.56586 ],
        [  47.24234 ,  277.8648  ,  -67.92094 ,  -29.76979 , -515.50555 ]],
       dtype=float32),
 array([[-5.3341082e+08,  2.2326338e+10, -1.7078427e+09,  9.2867616e+08,
          7.1499203e+10],
        [-8.9373350e+08,  3.7478150e+10, -2.8667259e+09,  1.5609989e+09,
          1.2002530e+11],
        [-1.2540562e+09,  5.2629967e+10, -4.0256092e+09,  2.1933217e+09,
          1.6855138e+11],
        [-1.6143789e+09,  6.7781779e+10, -5.1844925e+09,  2.8256445e+09,
          2.1707750e+11]], dtype=float32),
 array([[-inf,  inf,  inf, -inf, -inf],
        [-inf,  inf,  inf, -inf, -inf]], dtype=float32))